In [1]:
!pip install idx2numpy

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from keras.datasets import mnist



(training_data,training_labels),(test_data,test_labels) = mnist.load_data()

image_train = training_data[:60000]
image_test = test_data[:10000]

labels_train = training_labels[:60000]
labels_test = test_labels[:10000]

In [2]:
def img_blocks(img =list(list())  , block_size=tuple()):
    blocks = []
    for i in range(0, int(img.shape[0]/block_size[0])):
        for j in range(0, int(img.shape[1]/block_size[1])):
            blocks.append(img[block_size[0]*i:block_size[0]*(i+1), block_size[1]*j:block_size[1]*(j+1)])
    return blocks

def centriod(img =list(list())  , block_size=tuple()):
    blocks = img_blocks(img=img, block_size=block_size)
    img_tra = list()
    for block in blocks:
        x_c = float()
        y_c = float()
        num_x, num_y = 0 , 0
        denum = 0
        
        for x in range(0, block_size[0]):
            for y in range(0, block_size[1]):
                num_x   += x*block[x,y]
                num_y   += y*block[x,y]
                denum   += block[x,y]
        
        if denum==0:
            img_tra.append(0)
            img_tra.append(0)
        else:
            img_tra.append(num_x/denum)
            img_tra.append(num_y/denum)
        
    return img_tra
                
    
def train_centroid(imgs=image_train, block_size=(7,7)):
    train_centriod = list()
    for img in imgs:
        train_centriod.append(centriod(img=img, block_size=block_size))
    return np.array(train_centriod)

def test_centroid(imgs=image_test, block_size=(7,7)):
    test_centriod = list()
    for img in imgs:
        test_centriod.append(centriod(img=img, block_size=block_size))
    return np.array(test_centriod)


def argmax(vector):
    max = -1000
    ind=0
    for i in range(len(vector)):
        if(vector[i]>max):
            max=vector[i]
            ind=i
    return ind

In [3]:
train_centriod = train_centroid(imgs=image_train, block_size=(7,7))
test_centroid = test_centroid(imgs=image_test, block_size=(7,7))

In [4]:

# def calc_mean(data= train_centriod):
#     mean = []

#     for img in data:
#         sum=0
#         for i in range(data.shape[1]):
#             sum+=img[i]
#         mean.append(sum/data.shape[1])
#     return mean

# def calc_standart_deviation(data = train_centriod, mean =calc_mean(data= train_centriod)):
#     segma = []
#     for img in data:
#         sum=0
#         for i in range(data.shape[1]):
#             sum+=(img[i]-mean[i])**2

#         segma.append(np.sqrt(sum/data.shape[1]))
#     return segma
# def normallize(data=train_centriod):
#     imgs = data
#     mean = calc_mean(data)
    
#     segma = calc_standart_deviation(data,mean)
#     for img in data:
#         for i in range(data.shape[1]):
#             img[i]=(img[i]-mean[i])/segma[i]
            
#     return np.array(data), np.array(imgs)



# train, train_centriod = normallize(train_centriod)

# test, test_centroid= normallize(test_centroid)

In [5]:
train_centriod.shape

(60000, 32)

In [6]:

def normallize(data):
    mins=[]
    maxs=[]
    for i in range(data.shape[1]):
        min=1000
        max=-1000
        for img in data:
            if(img[i]>max):
                max=img[i]
            if(img[i]<min):
                min=img[i]
        maxs.append(max)
        mins.append(min)
        
    imgs = data
    for i in range(data.shape[1]):
        for img in data:
            img[i]=(img[i]-mins[i])/(maxs[i]-mins[i])
    
    return data


train = normallize(train_centriod)

test= normallize(test_centroid)

In [7]:
train_label = np.random.rand(labels_train.shape[0], 10)
for i in range(len(labels_train)):
    list= [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    list[labels_train[i]]=1
    train_label[i]=list
    
    
test_label = np.random.rand(labels_test.shape[0], 10)
for i in range(len(labels_test)):
    list= [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    list[labels_test[i]]=1
    test_label[i]=list


In [8]:
def MSE(y_true,y_predicted):
    return np.power(y_true - y_predicted, 2)*0.5

def partial_MSE(y_true,y_predicted):
    return (y_true - y_predicted) 

# def sigmoid (self,s,drivative=False): # Activation function
#     if(drivative == True):
#         return np.dot(s,(np.subtract(1,s).T))
#     return 1/(1 + np.exp(-s))

In [44]:


class Dense:
    
    
    def __init__(self,input_size,output_size):
        np.random.seed(1)
        self.weight = np.random.rand(output_size,input_size)-0.5
        self.delta = None
                  
                  
    def forward (self,input,weight):
        self.input = input
        product = np.dot(weight,self.input)
        self.output = np.array(1/(1 + np.exp(-product)))

        return np.array(self.output),np.array(self.weight)
                  
                  
#   Take the delta of the output layer
    def backward (self,delta,learning_rate = 0.01):
        if len(delta.shape)== 1:
            delta = np.array(delta).reshape(delta.shape[0], 1)
        if len(self.output.shape)== 1:
            self.output = np.array(self.output).reshape(self.output.shape[0], 1)

        self.weight += learning_rate * np.dot(delta,self.output.T)  
        self.delta = np.dot(np.dot(self.weight.T,delta),np.dot(self.output.T,np.subtract(1,self.output)))
        return self.delta


In [45]:
net = [
    Dense(32,58),
    Dense(58,10),
    Dense(10,10)
]

epoches = 1
for e in range(epoches):
    error = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    for x,y in zip(train,train_label):
        flag=True
        output = x
        weight = net[0].weight
        for layer in net:
            if layer == net[0]:
                layer.output = output
                continue
            output,weight = layer.forward(output,weight)
            
        
        delta = partial_MSE(y,output) #y_true - output
        

        for layer in reversed(net):

            if(flag == True):
                delta = np.dot(delta.T,np.dot(layer.output,(np.subtract(1,layer.output).T))).T  

            else :

                delta = layer.backward(delta)
            flag=False


In [46]:
pridect = []
for x,y in zip(test, labels_test):
    output = x
    weight = net[0].weight
    for layer in net:
        if layer == net[0]:
            layer.output = output
            continue
        output,weight = layer.forward(output,weight)
        
    pridect.append(argmax(output))
#     print(y, output.argmax(), sep='     ')

print(((labels_test == pridect).sum()/labels_test.shape[0])*100, sep='     ')

75.26


In [12]:
# epoches = 1000
# neural neteork =[dense(32,48),dense(48,30),.....dense(30,10)]
# for i in range(epoches)
#     for layer in neural network:
#         list.append(layer.forward())
#     error = error of output layer  wji from i to j // get the error of layer j 
#     for back:
#         gradient = backward(eroor,)0.01)


In [13]:


# net = [
#     Dense(32,16),
#     Dense(16,10),
#     Dense(10,5),
#     Dense(5,10),
#     Dense(10,10)
# ]

# epoches = 50

# for e in range(epoches):
#     error = 0
#     for x,y in zip(train_centriod,labels_train):
#         flag=True
#         output = x
#         weight = net[0].weight
#         for layer in net:
#             if layer == net[0]:
#                 layer.output = output
#                 continue
#             output,weight = layer.forward(output,weight)
        
#         print(y, output.argmax(),sep="      ")
#         error += MSE(y,output.argmax())
#         delta = partial_MSE(y,output) #y_true - output
#         cnt = len(net)
        
#         for layer in reversed(net):
#             if layer == net[0] :
#                 break
            
#             if(flag == True):
#                 delta = np.dot(delta.T,np.dot(layer.output,(np.subtract(1,layer.output).T))).T  
            
#             else :
                
#                 delta = layer.backward(delta)
#             flag=False
#     print(error/len(train_label))
        